In [ ]:
import fastdyn_fic_dmf as dmf
from scipy.signal import butter, lfilter
import numpy as np
import matplotlib.pyplot as plt
# Fetch default parameters
import tracemalloc
from scipy.io import loadmat
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_multitaper

def compute_fcd(data, wsize, overlap, isubdiag):
    T, N = data.shape
    win_start = np.arange(0, T - wsize - 1, wsize - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    print(fcd.shape)
    print(data.shape)
    print((data[win_start[2]:win_start[2] + wsize + 1, :]).shape)
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + wsize + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0],isubdiag[1]]
    return fcd


## Homeostatic grid seacrch


In [ ]:
nb_steps = 100000
C = loadmat('../SC_and_5ht2a_receptors.mat')['sc90']
C = 0.2*C/np.max(C)
N = C.shape[0]
brunout = 5
params = dmf.default_params(C=C)
params['seed'] = 2
G_max = 6
DECAY_max = 60000
OBJ_RATE_max = 15
#G_step = 0.1
#DECAY_step = 0.2
G_step = 2
DECAY_step = 20000
OBJ_RATE_step = 3
G_range = np.arange(0,G_max,G_step)
DECAY_range = [100,10000,60000]#np.arange(1,DECAY_max,DECAY_step)
OBJ_RATE_range = np.arange(1,OBJ_RATE_max,OBJ_RATE_step)
amount_slow_regions_grid = np.zeros((len(G_range), len(DECAY_range),len(OBJ_RATE_range)))
mean_slow_grid = np.zeros((len(G_range), len(DECAY_range),len(OBJ_RATE_range)))
std_slow_grid = np.zeros((len(G_range), len(DECAY_range),len(OBJ_RATE_range)))
homeostatic_fittness_grid = np.zeros((len(G_range), len(DECAY_range),len(OBJ_RATE_range)))
rates_grid,rates_inh_grid, bold_grid, fic_t_grid = np.zeros((len(G_range),len(DECAY_range),len(OBJ_RATE_range),N,nb_steps)),np.zeros((len(G_range),len(DECAY_range),len(OBJ_RATE_range),N,nb_steps)),np.zeros((len(G_range),len(DECAY_range),len(OBJ_RATE_range),N,nb_steps)),np.zeros((len(G_range),len(DECAY_range),len(OBJ_RATE_range),N,nb_steps))
for idx_G,G in enumerate(G_range):
    for idx_DECAY,DECAY in enumerate(DECAY_range):
        for idx_OBJ_RATE,OBJ_RATE in enumerate(OBJ_RATE_range):
            params['G'] = G
            params['taoj'] = DECAY
            params['obj_rate'] = OBJ_RATE
            #params['taoj'] = 210000
            params['J'] = 0.75*params['G']*params['C'].sum(axis=0).squeeze() + 1
            N = C.shape[0]
            tracemalloc.start()
            rates, rates_inh, bold, fic_t = dmf.run(params, nb_steps,return_rate=True, return_bold=True, return_fic=True)
            rates_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = rates
            rates_inh_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = rates_inh
            #bold_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = bold
            fic_t_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = fic_t
            # Slicing and computations
            bold = bold[:, (np.ceil(brunout / params['TR'])).astype(int):]
            rates = rates[:, np.ceil(brunout * 1000).astype(int):]
            rates_inh = rates_inh[:, np.ceil(brunout * 1000).astype(int):]
            fic_t = fic_t[:, np.ceil(brunout * 1000).astype(int):]
            sampling_freq = 10000  
            power_spectrum,frequencies = psd_array_multitaper(rates, sfreq=sampling_freq, fmin=0, fmax=100,bandwidth=32*(sampling_freq/rates.shape[1]))
            plow_ptot = np.sum(power_spectrum[:, :39], axis=1)/np.sum(power_spectrum[:,:], axis=1)
            amount_slow_regions = np.sum(plow_ptot>0.3)
            mean_slow = np.mean(plow_ptot)
            std_slow = np.std(plow_ptot)
            homeostatic_fittness =  OBJ_RATE - np.mean(rates)  #if not np.isnan(np.mean(rates)) else 0
            amount_slow_regions_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = amount_slow_regions
            mean_slow_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = mean_slow
            std_slow_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = std_slow
            homeostatic_fittness_grid[idx_G, idx_DECAY,idx_OBJ_RATE] = homeostatic_fittness

Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 DPSS windows
Calling run
Ending simulation     Using multitaper spectrum estimation with 30 D

In [ ]:
import matplotlib.pyplot as plt

for i, decay_value in enumerate(DECAY_range):
    fig, ax = plt.subplots()
    
    # Extracting the corresponding slice from homeostatic_fittness_grid
    slice_data = homeostatic_fittness_grid[i]
    
    # Plotting the heatmap
    heatmap = ax.imshow(slice_data, cmap='viridis', aspect='auto', vmin=np.min(homeostatic_fittness_grid), vmax=np.max(homeostatic_fittness_grid))
    plt.colorbar(heatmap, ax=ax)
    
    ax.set_xlabel('OBJ_RATE_rand')
    ax.set_ylabel('G_range')
    ax.set_title(f'Heatmap for DECAY_range = {decay_value}')
    
    plt.show()
